In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('alzheimer.csv')

In [3]:
df.head()

,Group,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,Nondemented,M,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,Nondemented,M,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,Demented,M,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,Demented,M,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,Demented,M,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import shap
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
# 🧹 Preprocessing
df = df.dropna()  # Remove rows with missing values

# Label encode all object (categorical) columns
for col in df.select_dtypes(include=['object']).columns:
    df[col] = LabelEncoder().fit_transform(df[col])

# Double-check
df.head()


,Group,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF,Group_encoded,patient_text
0,2,1,87,14,2.0,27.0,0.0,1987,0.696,0.883,2,314
1,2,1,88,14,2.0,30.0,0.0,2004,0.681,0.876,2,323
5,2,0,88,18,3.0,28.0,0.0,1215,0.710,1.444,2,321
6,2,0,90,18,3.0,27.0,0.0,1200,0.718,1.462,2,335
7,2,1,80,12,4.0,28.0,0.0,1689,0.712,1.039,2,227


In [5]:
df['M/F'] = df['M/F'].map({'M': 'Male', 'F': 'Female'})

In [6]:
label_encoder = LabelEncoder()
df['Group_encoded'] = label_encoder.fit_transform(df['Group'])  # Demented=0, Nondemented=1


In [7]:
def row_to_text(row):
    return (
        f"The patient is a {row['Age']} year old {row['M/F']} with {row['EDUC']} years of education. "
        f"Their socioeconomic status is {row['SES']}, MMSE score is {row['MMSE']}, and CDR is {row['CDR']}. "
        f"Brain volume metrics include eTIV of {row['eTIV']}, nWBV of {row['nWBV']}, and ASF of {row['ASF']}."
    )

df['patient_text'] = df.apply(row_to_text, axis=1)

In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(df['patient_text'].tolist(), show_progress_bar=True)


Batches: 100%|██████████| 12/12 [00:03<00:00,  3.84it/s]


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    embeddings,
    df['Group_encoded'],
    test_size=0.2,
    random_state=42,
    stratify=df['Group_encoded']
)


In [10]:
clf = xgb.XGBClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=5,
    eval_metric='logloss',
    use_label_encoder=False
)
clf.fit(X_train, y_train)



/Users/udaysharma/Desktop/akrit/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:39:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=150,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [11]:
y_pred = clf.predict(X_test)


In [12]:
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

print(f"✅ Accuracy: {accuracy_score(y_test, y_pred):.4f}")


📊 Classification Report:
              precision    recall  f1-score   support

   Converted       0.00      0.00      0.00         7
    Demented       0.78      0.81      0.79        26
 Nondemented       0.79      0.89      0.84        38

    accuracy                           0.77        71
   macro avg       0.52      0.57      0.54        71
weighted avg       0.71      0.77      0.74        71

✅ Accuracy: 0.7746
